# Verbatim mode

In [1]:
from braket.circuits import Circuit, Observable, Gate, Noise
from braket.aws import AwsDevice
import numpy as np
import datetime

In [3]:
# device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1")
# device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-11")
device = AwsDevice("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")

In [4]:
# Available gates
for gate in device.properties.paradigm.nativeGateSet:
    print(gate)

ecr
i
rz
v
x


In [5]:
# verbatim circuit 
identity = Circuit().rz(0,np.pi).rz(0,-np.pi)
circ1 = Circuit().add_verbatim_box(identity)
print(circ1)

# verbatim circuit with observables
identity = Circuit().rz(0,np.pi).rz(0,-np.pi)
circ2 = Circuit().add_verbatim_box(identity)
circ2.expectation(Observable.X(), target=[0])
circ2.expectation(Observable.X(), target=[1])
print(circ2)

T  : |      0      |   1    |    2    |     3     |
                                                   
q0 : -StartVerbatim-Rz(3.14)-Rz(-3.14)-EndVerbatim-

T  : |      0      |   1    |    2    |     3     |
T  : |      0      |   1    |    2    |     3     | Result Types |
                                                                  
q0 : -StartVerbatim-Rz(3.14)-Rz(-3.14)-EndVerbatim-Expectation(X)-
      |                                |                          
q1 : -*************--------------------***********-Expectation(X)-

T  : |      0      |   1    |    2    |     3     | Result Types |


### Execute circuits

In [6]:
# For OQC's device
# reference: https://github.com/aws/amazon-braket-schemas-python-staging/blob/main/src/braket/task_result/oqc_metadata_v1.py
result1 = device.run(circ1, shots=100, disable_qubit_rewiring=True).result()
compiled_program = result1.additional_metadata.oqcMetadata.executedProgram
print(compiled_program)
print(result1.measurement_counts)


OPENQASM 2.0;
include "qelib1.inc";
qreg q[1];
creg b[1];

rz(3.141592653589793) q[0];
rz(-3.141592653589793) q[0];
measure q -> b;
Counter({'0': 99, '1': 1})


In [7]:
result2 = device.run(circ2, shots=100, disable_qubit_rewiring=True).result()
compiled_program = result2.additional_metadata.oqcMetadata.executedProgram
print(compiled_program)
print(result2.measurement_counts)
print(result2.values)


OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg b[2];

rz(3.141592653589793) q[0];
rz(-3.141592653589793) q[0];
h q[0];
h q[1];
measure q -> b;
Counter({'01': 30, '00': 26, '11': 26, '10': 18})
[0.12, -0.12]


In [ ]:
# For Rigetti's device
# result = device.run(circ1, s3_folder, shots=100, disable_qubit_rewiring=True).result()
# compiled_program = result.additional_metadata.rigettiMetadata.compiledProgram
# print(compiled_program)
# print(result.measurement_counts)